In [14]:
import numpy as np
import pandas as pd
from functions import fetch_data

In [15]:
df1 = fetch_data('fb_df')
df2 = fetch_data('aapl_df')
df3 = fetch_data('nflx_df')
df4 = fetch_data('goog_df')
df5 = fetch_data('sp500_df')

In [16]:
class Portfolio:
    
    """For each trade, the object will buy or sell positions in the portfolio according how much is available. Passing
    0.5 to the object means that it will purchase 50% of all shares available (according to current balance and price). 
    Passing -0.5 to the object means that it will sell 50% of all shares available (according to how many it has). Sales
    are executed first, then purchase quantities are normalized according to the balance after the sales."""
    
    def __init__(self, stocks, balance_init=1_000_000, fee=0.001):
        
        assert isinstance(stocks, list) and len(stocks) >= 1
        assert balance_init >= 100
        assert fee >= 0
        
        self.stocks = [stock.lower() for stock in stocks]
        self.balance_init = balance_init
        self.fee = fee
        
    def reset(self):
        
        positions = {pos:0.0 for pos in self.stocks}
        self.positions_full = positions.copy() # Number of shares held for each stock
        self.positions_norm = positions.copy() # Portfolio exposure of each stock
        self.positions_norm['_out'] = 1.0 # Portfolio exposure includes none

        self.balance = self.balance_init # Liquidity in account
        self.net_worth = self.balance_init # liquidity + shares*prices
        
        self.days_passed = 0
        self.profits = []
        
        return self
        
    def make_trade(self, actions, prices):
        
        step = 1e-5
        assert isinstance(actions, dict)
        assert isinstance(prices, dict)
        
        net_worth_prev = self.net_worth
        sales = {stock:action for stock, action in actions.items() if action < 0}
        purchases = {stock:action for stock, action in actions.items() if action > 0}
        
        # Execute sales first
        for stock, action in sales.items():
            
            # How many shares are held
            total_possible = self.positions_full[stock]
            # Sell the specified portion of available held
            shares_sold = total_possible * -action
            # Profit is the price times quantity minus fee
            profit = shares_sold * prices[stock] * (1 - self.fee)

            print('sold', round(shares_sold, 1), 'shares of', stock)
            self.positions_full[stock] -= shares_sold
            self.balance += profit
                
        # Adjust purchase allocations if necessary   
        if sum(purchases.values()) > 1:
            purchases = {
                k : v/sum(purchases.values())
                for k, v in purchases.items()
            }
  
        # Execute purchases
        balance = self.balance
        for stock, action in purchases.items():    
            
            # How many shares can be afforded
            total_possible = balance / (prices[stock]*(1+self.fee))
            # Buy specified amount of available shares
            shares_bought = total_possible * action
            # Cost is th eprices times the quantity plus fee
            cost = shares_bought * prices[stock] * (1 + self.fee)
            
            print('bought', round(shares_bought, 1), 'shares of', stock)
            self.positions_full[stock] += shares_bought
            self.balance -= cost

        # Calculate net_worth
        self.net_worth = self.balance + sum(
            shares*price for shares, price 
            in zip(self.positions_full.values(), prices.values())
        )
        
        # Calculate exposures
        for position in self.positions_norm.keys():
            if position == '_out':
                self.positions_norm[position] = self.balance / self.net_worth
            else:
                self.positions_norm[position] = (self.positions_full[position]*prices[position]) / self.net_worth
                
        self.days_passed += 1
        self.profits.append(self.net_worth-net_worth_prev)
        
    def report(self):
        
        print('Balance:', round(self.balance, 5))
        print('Net worth:', self.net_worth)
        print('Shares held:', self.positions_full)
        print('Exposures:', self.positions_norm, '|', round(sum(self.positions_norm.values()), 5))
        
        try:
            print('Current profit:', self.profits[-1])
        except IndexError:
            print("Current profit: NA")
       
        try:
            print('Average profit:', sum(self.profits)/self.days_passed)
        except ZeroDivisionError:
            print("Average profit: NA")
       
        print('Total profit:', sum(self.profits))
        print('n Steps:', self.days_passed)
            
        return sum(self.profits)
    

---

In [27]:
purchases

{'a': 0.6923076923076923, 'b': 0.15384615384615385, 'c': 0.15384615384615385}

In [19]:
portfolio = Portfolio(['aapl', 'NFLX', 'Goog'])
portfolio.reset();

In [20]:
portfolio.report();

Balance: 1000000
Net worth: 1000000
Shares held: {'aapl': 0.0, 'nflx': 0.0, 'goog': 0.0}
Exposures: {'aapl': 0.0, 'nflx': 0.0, 'goog': 0.0, '_out': 1.0} | 1.0
Current profit: NA
Average profit: NA
Total profit: 0
n Steps: 0


In [21]:
allocations = {'aapl':1, 'nflx':1, 'goog':1}
stocks = {
    'aapl': float(df2[df2['date']=='2012-05-21']['close']), 
    'nflx': float(df3[df3['date']=='2012-05-21']['close']),
    'goog': float(df4[df4['date']=='2012-05-21']['close']),
    }

portfolio.make_trade(allocations, stocks)

bought 16612.0 shares of aapl
bought 32492.4 shares of nflx
bought 1088.6 shares of goog


In [22]:
portfolio.report();

Balance: -0.0
Net worth: 999000.999000999
Shares held: {'aapl': 16612.047324050436, 'nflx': 32492.367277382677, 'goog': 1088.562289365722}
Exposures: {'aapl': 0.33333333333333337, 'nflx': 0.33333333333333337, 'goog': 0.33333333333333337, '_out': -1.1653173714876175e-16} | 1.0
Current profit: -999.0009990009712
Average profit: -999.0009990009712
Total profit: -999.0009990009712
n Steps: 1


In [23]:
allocations = {'aapl':0.6, 'nflx':0.7, 'goog':-0.5}
stocks = {
    'aapl': float(df2[df2['date']=='2012-05-22']['close']), 
    'nflx': float(df3[df3['date']=='2012-05-22']['close']),
    'goog': float(df4[df4['date']=='2012-05-22']['close']),
    }

portfolio.make_trade(allocations, stocks)

sold 544.3 shares of goog
bought 3771.9 shares of aapl
bought 9049.6 shares of nflx


In [24]:
portfolio.report();

Balance: -0.0
Net worth: 970194.8658505111
Shares held: {'aapl': 20383.979920125195, 'nflx': 41541.9748279463, 'goog': 544.281144682861}
Exposures: {'aapl': 0.4179302540711274, 'nflx': 0.41417409651623394, 'goog': 0.16789564941263868, '_out': -1.499896128146387e-17} | 1.0
Current profit: -28806.133150487905
Average profit: -14902.567074744438
Total profit: -29805.134149488877
n Steps: 2


In [25]:
allocations = {'aapl':-0.33, 'nflx':-0.33, 'goog':0.8}
stocks = {
    'aapl': float(df2[df2['date']=='2012-05-23']['close']), 
    'nflx': float(df3[df3['date']=='2012-05-23']['close']),
    'goog': float(df4[df4['date']=='2012-05-23']['close']),
    }

portfolio.make_trade(allocations, stocks)

sold 6726.7 shares of aapl
sold 13708.9 shares of nflx
bought 730.8 shares of goog


In [26]:
portfolio.report();

Balance: 55524.47328
Net worth: 1006861.6844436277
Shares held: {'aapl': 13657.26654648388, 'nflx': 27833.12313472402, 'goog': 1275.1171499091984}
Exposures: {'aapl': 0.27639949364378946, 'nflx': 0.2839771323549854, 'goog': 0.3844772957048025, '_out': 0.0551460782964227} | 1.0
Current profit: 36666.81859311659
Average profit: 2287.2281478759055
Total profit: 6861.684443627717
n Steps: 3
